# Метод простой итерации для системы линейных уравнений

Для начала создадим случайную матрицу с диагональным доминированием.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Итерации Якоби

Задано

$$
A x = b
$$

Отделим диагональную часть $D$,

$$ A = D + (A - D) $$

и напишем

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

далее зациклим

$$
x_{n + 1} = B x_{n} + c\;,
$$

где

$$
B = D^{-1} (A - D) \qquad \text{и} \qquad c = D^{-1} b
$$


Создадим матрицу и правую часть уравнения для итераций Якоби

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# проверка на адекватность
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx есть точное решение, вычислим его, используя прямой метод
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Проверьте, что $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Выполните итерации Якоби

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Проверка результата:

A @ x - b

array([  1.11022302e-16,   1.11022302e-16,  -1.11022302e-16,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -1.38777878e-17,   0.00000000e+00,   2.77555756e-17,
         1.11022302e-16])

### Task I.1

Используя отдельные элементы выше, составьте единую функцию, реализующую итерацию Якоби. Такая функция должна получать матрицу $A$, вектор $b$ и количество итераций для цикла. 

Матрица $A$ в иллюстрации выше диагонально доминирующая по построению.
Что произойдет, если уменьшить значения диагональных элементов $A$? Проверьте сходимость итерации Якоби, а также значение нормы $B$.

(20% итоговой оценки)


In [ ]:
# ... ENTER YOUR CODE HERE ...

# II. Метод итераций Зейделя.

##### Задание II.1

Напишите алгоритм, использующий итерации Зейделя. 

Проверьте на случайной матрице. Проверьте сходимость.

(30% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...

# III. Схема минимальных невязок.

### Задание III.1

Реализуйте "метод минимальных невязок": явный нестационарный метод, который на каждом шаге выбирает параметр итерации $\tau_n$, чтобы минимизировать невязку $\mathbf{r}_{n+1}$ с заданным $\mathbf{r}_n$. Протестируйте алгоритм на случайной матрице, проверьте сходимость к решению в терминах нормы невязки и отклонения от точного решения (который вы можете получить прямым методом). Изучите, как параметр $\tau_n$ влияет на итерационный процесс.

(50% итоговой оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...